## LightFM recommendation



In [1]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from itertools import product

In [2]:
train_purchases_df = pd.read_csv("../data/dressipi_recsys2022/train_purchases.csv", parse_dates=['date'])
train_sessions_df = pd.read_csv("../data/dressipi_recsys2022/train_sessions.csv" , parse_dates=['date'])
item_features_df = pd.read_csv("../data/dressipi_recsys2022/item_features.csv")
candidate_items_df = pd.read_csv("../data/dressipi_recsys2022/candidate_items.csv")


# Split the dataset

In [3]:
# Test set is
train_p_df, test_p_df = train_test_split(train_purchases_df, test_size=0.02)
#test_sessions_set = set(test_p_df['session_id'])

#train_s_df = train_sessions_df[~train_sessions_df['session_id'].isin(test_sessions_set)]
#test_s_df = train_sessions_df[train_sessions_df['session_id'].isin(test_sessions_set)]

#print(f"len(test_sessions_set)={len(test_sessions_set)}")
print(f"len(train_p_df)={len(train_p_df)} train purchases")
print(f"len(test_p_df)={len(test_p_df)}   test purchases")
#print(f"len(train_s_df)={len(train_s_df)} train sessions")
#print(f"len(test_s_df)={len(test_s_df)}   test sessions")



len(train_p_df)=980000 train purchases
len(test_p_df)=20000   test purchases


## The task is to submit a csv that has 100 ranked predictions for each query session.
Header and columns as in the example below. Header is required. The order of rows does not matter for the evaluation system but we recommend to sort the file by session_id and rank for easier manual inspection. 
<pre>
session_id,item_id,rank
1,100,1
1,105,2
1,107,3
...
1,101,100
2,108,1
2,107,2
...
</pre>

In [4]:
pip install lightfm

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Calculate Mrr (Mean reciprocal rank)
def calc_mrr(result_df, test_df):
    mrr = 0
    # Iterate all sessions
    for sess_id in test_df['session_id']:
        # Make view for only this session with all ranked
        ranked = result_df[result_df['session_id']==sess_id]['item_id'].reset_index(drop=True)
        real_item_id = test_df[test_df['session_id']==sess_id]['item_id'].reset_index(drop=True)[0] 
        first_rank = 100
        found_t = ranked[ranked == real_item_id]
        if len(found_t)!=0 :
            first_rank = found_t.index[0]+1
            #print('first_rank=', first_rank)
        mrr =mrr+ 1/first_rank
        
    mrr = mrr / test_df['session_id'].nunique()
    return mrr
    
    
    

In [6]:
# Test of method is commented
#calc_mrr(pd.DataFrame.from_dict({'session_id':[1,1,2,2,2,3,3,3], 'item_id':[1,2,3,4,5,6,7,8]}),\
#                     pd.DataFrame.from_dict({'session_id':[1,2,3], 'item_id':[2,5,8]}))
#(1/2 + 1/3 + 1/100)/3

In [7]:

from lightfm.data import Dataset
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

## Start building the feature for light FM
User features are of the form tuple iterable tuple (session_id, [month, hour])

In [8]:
train_sessions_df['month']='m' + train_sessions_df['date'].dt.month.apply(str)
train_sessions_df['hour']='h' + train_sessions_df['date'].dt.hour.apply(str)

# Aggregate by sessionId and add hour and month as list of features
user_features = train_sessions_df.groupby('session_id').agg(
    session_id=pd.NamedAgg(column="session_id", aggfunc="min"),
    month=pd.NamedAgg(column="month", aggfunc=set),
    hour=pd.NamedAgg(column="hour", aggfunc=set),
).reset_index(drop=True).apply(lambda row: (row['session_id'], list(row['month']) + (list(row['hour']))), axis = 1)

user_features.head(2)

0    (3, [m12, h21])
1    (13, [m3, h19])
dtype: object

Item features are tuples of this form: iterable tuple (item_id, [category1-value1, category2-value2])

In [9]:
item_features_df['cat_val'] = item_features_df['feature_category_id'].apply(str)+ '-' + item_features_df['feature_value_id'].apply(str)

# Aggregate by item_id and add category and value as list of features
item_features = item_features_df.groupby('item_id').agg(
    item_id=pd.NamedAgg(column="item_id", aggfunc="min"),
    cat_val=pd.NamedAgg(column="cat_val", aggfunc=list),
).reset_index(drop=True).apply(lambda row: (row['item_id'], row['cat_val']), axis = 1) 
item_features.head(2)

0    (2, [56-365, 62-801, 68-351, 33-802, 72-75, 29...
1    (3, [56-365, 69-592, 68-14, 17-378, 32-902, 11...
dtype: object

Building the interactions of the form: iterable tuple (session_id, item_id, weight)

In [93]:
train_sessions_df['score'] = train_sessions_df.groupby(['session_id']).cumcount()+1
train_sessions_df['views'] = train_sessions_df.groupby(['session_id'])['item_id'].count()
train_sessions_df[(train_sessions_df['item_id']==2447)&(train_sessions_df['session_id']==4254727)]

,session_id,item_id,date,month,hour,score,views,views1
4544540,4254727,2447,2021-03-11 14:11:29.645,m3,h14,3,NaN,NaN


In [79]:
# Give 0.3 weight for just view in the session
# and 1.0 for a purchase
train_interactions1 = train_sessions_df.apply(
    lambda row: (row['session_id'], row['item_id'], 0.3), axis = 1)  
train_interactions2 = train_p_df.apply(
    lambda row: (row['session_id'], row['item_id'], 1), axis = 1) 

train_interactions = pd.concat([train_interactions1, train_interactions2])

In [11]:
#train_interactions.reset_index(inplace=True)
#print(f"train_interactions.info() : {train_interactions.info()}")
train_interactions.head(2)

0    (3, 9655, 0.3)
1    (3, 9655, 0.3)
dtype: object

## Build Light FM Dataset

In [12]:
from collections import Counter
item_features_counter = Counter()
item_features.map(lambda x: item_features_counter.update(x[1]))
item_features_vals = list(item_features_counter.keys())

user_features_counter = Counter()
user_features.map(lambda x: user_features_counter.update(x[1]))
user_features_vals = list(user_features_counter.keys())

print(f"User feature values: {user_features_vals[:30]}...")
print(f"Item feature values: {item_features_vals[:30]}...")

User feature values: ['m12', 'h21', 'm3', 'h19', 'm8', 'm11', 'h16', 'm2', 'h17', 'h18', 'm5', 'h12', 'm4', 'm6', 'h10', 'h15', 'h20', 'h11', 'm9', 'm1', 'h6', 'h13', 'h14', 'm10', 'h8', 'h9', 'h7', 'h1', 'h0', 'm7']...
Item feature values: ['56-365', '62-801', '68-351', '33-802', '72-75', '29-123', '16-38', '50-76', '61-462', '53-6', '7-394', '69-885', '47-123', '69-592', '68-14', '17-378', '32-902', '11-859', '45-559', '7-452', '19-254', '46-825', '61-706', '73-544', '55-129', '63-861', '50-240', '59-180', '4-618', '5-605']...


In [13]:
import lightfm
from lightfm import cross_validation


ds = Dataset()
ds.fit( \
       train_sessions_df['session_id'].unique(), \
       item_features_df['item_id'].unique(), \
       item_features=item_features_vals, \
       user_features=user_features_vals)
item_features_ds                  = ds.build_item_features(item_features)
train_interactions_ds, weights_ds = ds.build_interactions(train_interactions)
user_features_ds                  = ds.build_user_features(user_features)

(trn_interactions_ds, tst_interactions_ds) = \
    lightfm.cross_validation.random_train_test_split(train_interactions_ds, test_percentage=0.2, random_state=None)

## Train the model

In [14]:
model = LightFM(
    no_components=10,
    learning_rate=0.02,
    loss='warp',
    random_state=42)

model.fit(
    trn_interactions_ds,
    item_features=item_features_ds,
    user_features=user_features_ds, 
    epochs=35, num_threads=4, verbose=True)

(user_map, feature_map, item_map, item_feature_map) = ds.mapping()

Epoch: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [13:16<00:00, 22.76s/it]


In [24]:
def predict_ratings(model, sessions_or_purchase_df, items_to_predict) :
    session_ids = pd.Series(sessions_or_purchase_df['session_id'].unique())
    all_users = session_ids \
        .apply(lambda x: user_map[x]).to_numpy()
    all_available_items = items_to_predict.apply(lambda x: item_map[x]).to_numpy()
    users =[]
    items =[]
    for user_item_tuple in product(all_users, all_available_items):
        users.append(user_item_tuple[0])
        items.append(user_item_tuple[1])
    preds = model.predict(np.array(users), np.array(items))
    
    session_ids_expanded = []
    item_ids_expanded = []
    for tup in product(session_ids, items_to_predict):
        session_ids_expanded.append(tup[0])
        item_ids_expanded.append(tup[1])
    
    
    df_score = pd.DataFrame({'session_id': np.array(session_ids_expanded), \
                      'item_id':np.array(item_ids_expanded) , \
                      'score':np.array(preds)})
    return df_score

In [72]:
# Determine which items to predict
# For the submission we load from "candidate_items.csv" but for
# initial testing we must take them from train_sessions.

#items_to_predict_array = train_sessions_df['item_id'].unique()

# Recommend from most sold items
items_to_predict_array = \
     list(train_sessions_df['item_id'].value_counts()[0:2000].keys())

print('Items to predict: ', len(items_to_predict_array))

Items to predict:  2000


In [76]:
t = predict_ratings(model, test_p_df[0:20], \
                    pd.Series(items_to_predict_array))
t1 = t.sort_values(['score'],ascending=False)

In [77]:
# DataFrameGroupBy: The function passed to apply must take a dataframe 
# as its first argument and return a DataFrame
# From the other side "cumcount()" is used to count in the group
t1['rank'] = t1.groupby('session_id').cumcount()+1


result_df = t1[t1['rank']<=100]#.drop('score', axis=1)
result_df.reset_index(inplace=True)
result_df.drop(['index'], inplace=True, axis=1)

result_df = result_df.sort_values( \
        by=['session_id', 'rank'],ascending=True)
#result_df.reset_index(inplace=True)
result_df.to_csv("temp_user_features.csv")
result_df



/home/bobi/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,session_id,item_id,score,rank
572,398713,14143,2.688654,1
683,398713,9185,2.433117,2
771,398713,21215,2.273204,3
791,398713,25007,2.242367,4
848,398713,11491,2.150226,5
...,...,...,...,...
1954,4261173,6817,0.980572,96
1956,4261173,18623,0.980331,97
1957,4261173,27695,0.979393,98
1959,4261173,563,0.971354,99


In [78]:
#test_p_df[test_p_df['session_id'].isin({2692943,1211445})]

calc_mrr(result_df, test_p_df)

0.009999999999998154

In [50]:
test_p_df.to_csv("test_p_df.csv")